### Import libraries

In [6]:
import pandas as pd

### Download data

In [5]:
# !curl -o './data/movie-data.zip' 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
# !unzip './data/movie-data.zip' -d './data'

Archive:  ./data/movie-data.zip
   creating: ./data/ml-latest-small/
  inflating: ./data/ml-latest-small/links.csv  
  inflating: ./data/ml-latest-small/tags.csv  
  inflating: ./data/ml-latest-small/ratings.csv  
  inflating: ./data/ml-latest-small/README.txt  
  inflating: ./data/ml-latest-small/movies.csv  


### Read data

In [7]:
movies = pd.read_csv('./data/ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings = pd.read_csv('./data/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [17]:
movies[movies.duplicated(subset=['movieId'])]

,movieId,title,genres


In [16]:
ratings[ratings.duplicated(subset=['userId', 'movieId'])]

,userId,movieId,rating,timestamp


In [37]:
movies_genre = (
    movies.merge(
        movies.genres.str.split(pat='|', expand=True),
        how='inner',
        left_index=True,
        right_index=True
    )
    .drop(columns=['genres'])
    .melt(
        id_vars=['movieId', 'title'],
        value_vars=range(0,10),
        var_name='i',
        value_name='genre'
    )
    .pivot_table(
        index=['movieId', 'title'],
        columns=['genre'],
        values='i',
        aggfunc='nunique'
    )
    .fillna(0)
)
movies_genre.head()

,genre,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,title,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Jumanji (1995),0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Grumpier Old Men (1995),0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Waiting to Exhale (1995),0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,Father of the Bride Part II (1995),0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
